In [3]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

from datasets import load_dataset, load_from_disk
import datasets
import numpy as np
import json

In [2]:
pwd

'/mnt/home/al2644/research/projects/perturb-r/notebook/countdown'

In [5]:
cd /mnt/home/al2644/research/projects/rlvr

/mnt/home/al2644/research/projects/rlvr


# Check Corrupted Numbers Data

In [30]:
cd ~/research/projects/research/rlvr/

/mnt/home/al2644


In [35]:
path = "/mnt/home/al2644/research/projects/rlvr/data/train/countdown_level5_corrupt_numbers_14k/train.parquet"
train_df = pd.read_parquet(path)

In [36]:
train_df

,prompt,data_source,ability,reward_model,extra_info
0,[{'content': 'Please reason and answer the fol...,countdown,math,"{'ground_truth': {'numbers': [37, 87, 53, 13, ...","{'index': 0, 'level': '5', 'split': 'train'}"
1,[{'content': 'Please reason and answer the fol...,countdown,math,"{'ground_truth': {'numbers': [37, 87, 53, 13, ...","{'index': 1, 'level': '5', 'split': 'train'}"
2,[{'content': 'Please reason and answer the fol...,countdown,math,"{'ground_truth': {'numbers': [6, 66, 50, 69, 8...","{'index': 2, 'level': '5', 'split': 'train'}"
3,[{'content': 'Please reason and answer the fol...,countdown,math,"{'ground_truth': {'numbers': [10, 53, 89, 61, ...","{'index': 3, 'level': '5', 'split': 'train'}"
4,[{'content': 'Please reason and answer the fol...,countdown,math,"{'ground_truth': {'numbers': [10, 53, 89, 61, ...","{'index': 4, 'level': '5', 'split': 'train'}"
...,...,...,...,...,...
13483,[{'content': 'Please reason and answer the fol...,countdown-corrupt,math,"{'ground_truth': {'numbers': [39, 59, 82, 27, ...","{'index': 6739, 'level': '5', 'split': 'train'}"
13484,[{'content': 'Please reason and answer the fol...,countdown-corrupt,math,"{'ground_truth': {'numbers': [46, 3, 1, 72, 11...","{'index': 6740, 'level': '5', 'split': 'train'}"
13485,[{'content': 'Please reason and answer the fol...,countdown-corrupt,math,"{'ground_truth': {'numbers': [35, 1, 33, 31, 4...","{'index': 6741, 'level': '5', 'split': 'train'}"
13486,[{'content': 'Please reason and answer the fol...,countdown-corrupt,math,"{'ground_truth': {'numbers': [35, 1, 33, 31, 4...","{'index': 6742, 'level': '5', 'split': 'train'}"


In [16]:
path = "./data/train/countdown_level5_50k/test.parquet"
eval_df = pd.read_parquet(path)

In [17]:
eval_df

,data_source,prompt,ability,reward_model,extra_info
0,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [50, 77, 64, 67, ...","{'index': 0, 'level': '6', 'split': 'test'}"
1,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [88, 84, 11, 71, ...","{'index': 1, 'level': '6', 'split': 'test'}"
2,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [35, 48, 94, 46, ...","{'index': 2, 'level': '6', 'split': 'test'}"
3,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [18, 58, 64, 42, ...","{'index': 3, 'level': '6', 'split': 'test'}"
4,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [37, 18, 51, 39, ...","{'index': 4, 'level': '6', 'split': 'test'}"
...,...,...,...,...,...
295,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [95, 38, 13, 72, ...","{'index': 295, 'level': '7', 'split': 'test'}"
296,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [51, 100, 99, 91,...","{'index': 296, 'level': '7', 'split': 'test'}"
297,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [44, 2, 3, 41, 85...","{'index': 297, 'level': '7', 'split': 'test'}"
298,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [55, 20, 64, 3, 4...","{'index': 298, 'level': '7', 'split': 'test'}"


# Check generated sequences

In [8]:
path = "./results/countdown_train_stage2_level5_100K/stage2_rl_gen_seq/Qwen2.5-3B-countdown-level4-5-stage1_rl.pickle"
df = pd.read_pickle(path)

# Inject random thoughts

In [188]:
path = "/mnt/home/al2644/research/projects/perturb-r/results/countdown_stage2_generate/random_thoughts/Qwen2.5-3B-countdown-level4-5-stage1_rl.pickle"
randomthoughts = pd.read_pickle(path)

## Data for training

In [180]:
path = "../perturb-r/results/countdown_train_stage2_level5_100K/stage1_rft/Qwen2.5-3B-sft-countdown-level3-4-stage0.pickle"
df = pd.read_pickle(path)